In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
import random

#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalize pixel values
X_train = X_train.astype('float32') / 245.0
X_test  = X_test.astype('float32') / 245.0

# One-hot encode labels
num_classes = 10
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat  = to_categorical(y_test, num_classes)

class_names = [
    'Airplane', 'Car', 'Bird', 'Cat', 'Deer',
    'Dog', 'Frog', 'Horse', 'Ship', 'Truck'
]

# CNN Model WITH Pooling
def model_with_pooling():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu',
               input_shape=(32, 32, 3), padding='same'),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


modelA = model_with_pooling()
historyA = modelA.fit(
    X_train, y_train_cat,
    epochs=10,
    validation_split=0.2,
    batch_size=64,
    verbose=0
)

lossA, accA = modelA.evaluate(X_test, y_test_cat, verbose=0)
print("Accuracy With Pooling:", accA)

# CNN Model WITHOUT Pooling
def model_without_pooling():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu',
               input_shape=(32, 32, 3), padding='same'),
        Conv2D(32, (3, 3), activation='relu', padding='same'),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


modelB = model_without_pooling()
historyB = modelB.fit(
    X_train, y_train_cat,
    epochs=10,
    validation_split=0.2,
    batch_size=64,
    verbose=0
)

lossB, accB = modelB.evaluate(X_test, y_test_cat, verbose=0)
print("Accuracy Without Pooling:", accB)


# Accuracy Comparison

print("\n" + "="*40)
print("ACCURACY COMPARISON")
print("="*40)
print(f"Model A (With Pooling) Accuracy: {accA:.4f}")
print(f"Model B (Without Pooling) Accuracy: {accB:.4f}")
print(f"Difference: {abs(accA - accB):.4f}")

if accA > accB:
    print(f"Model with Pooling performs better by {accA - accB:.4f}")
elif accB > accA:
    print(f"Model without Pooling performs better by {accB - accA:.4f}")
else:
    print("Both models have the same accuracy")


  #Accuracy & Loss Comparison Plots
plt.figure(figsize=(15, 5))

# Plot 1: Accuracy Bar Chart
plt.subplot(1, 3, 1)
models = ['With Pooling', 'Without Pooling']
accuracies = [accA, accB]
colors = ['skyblue', 'lightcoral']

bars = plt.bar(models, accuracies, color=colors,
                edgecolor='black', linewidth=2)
plt.ylabel('Accuracy', fontsize=12)
plt.title('CNN Accuracy Comparison', fontsize=14, fontweight='bold')
plt.ylim([0, 1])
plt.grid(True, alpha=0.3, axis='y')

for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2,
             bar.get_height(),
             f'{acc:.3f}',
             ha='center', va='bottom',
             fontweight='bold')


# Plot 2: Training Accuracy
plt.subplot(1, 3, 2)
plt.plot(historyA.history['accuracy'], label='With Pooling (Train)', linewidth=2)
plt.plot(historyA.history['val_accuracy'], label='With Pooling (Val)', linestyle='--', linewidth=2)
plt.plot(historyB.history['accuracy'], label='Without Pooling (Train)', linewidth=2)
plt.plot(historyB.history['val_accuracy'], label='Without Pooling (Val)', linestyle='--', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Training Accuracy Comparison', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim([0, 1])


# Plot 3: Training Loss
plt.subplot(1, 3, 3)
plt.plot(historyA.history['loss'], label='With Pooling (Train)', linewidth=2)
plt.plot(historyA.history['val_loss'], label='With Pooling (Val)', linestyle='--', linewidth=2)
plt.plot(historyB.history['loss'], label='Without Pooling (Train)', linewidth=2)
plt.plot(historyB.history['val_loss'], label='Without Pooling (Val)', linestyle='--', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Loss Comparison', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.suptitle('CIFAR-10 CNN Models: With vs Without Pooling Layers',
             fontsize=16, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()


# Random Image Prediction

print("\n" + "="*40)
print("RANDOM IMAGE PREDICTION")
print("="*40)

random_idx = random.randint(0, len(X_test) - 1)
test_image = X_test[random_idx]
true_label = y_test[random_idx][0]

predA = modelA.predict(test_image.reshape(1, 32, 32, 3), verbose=0)
predB = modelB.predict(test_image.reshape(1, 32, 32, 3), verbose=0)

pred_class_A = np.argmax(predA[0])
pred_class_B = np.argmax(predB[0])

confidence_A = predA[0][pred_class_A]
confidence_B = predB[0][pred_class_B]


# Display Results
plt.figure(figsize=(12, 5))

plt.subplot(1, 3, 1)
plt.imshow(test_image)
plt.title(f"Test Image\nTrue Class: {class_names[true_label]}",
          fontsize=12, fontweight='bold')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.barh(class_names, predA[0], color='skyblue', edgecolor='black')
plt.xlabel('Confidence')
plt.title(f"Model A (With Pooling)\nPrediction: {class_names[pred_class_A]}\n"
          f"Confidence: {confidence_A:.2%}",
          fontsize=12, fontweight='bold')
plt.xlim([0, 1])
plt.grid(True, alpha=0.3, axis='x')

plt.subplot(1, 3, 3)
plt.barh(class_names, predB[0], color='lightcoral', edgecolor='black')
plt.xlabel('Confidence')
plt.title(f"Model B (Without Pooling)\nPrediction: {class_names[pred_class_B]}\n"
          f"Confidence: {confidence_B:.2%}",
          fontsize=12, fontweight='bold')
plt.xlim([0, 1])
plt.grid(True, alpha=0.3, axis='x')

plt.suptitle('Random Image Prediction Comparison',
             fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()



# Detailed Prediction Output

print(f"True Label: {class_names[true_label]}")

print("\nModel A (With Pooling):")
print(f" Predicted: {class_names[pred_class_A]} "
      f"with {confidence_A:.2%} confidence")
print(f" Correct: {'✓' if pred_class_A == true_label else '✗'}")

print("\nModel B (Without Pooling):")
print(f" Predicted: {class_names[pred_class_B]} "
      f"with {confidence_B:.2%} confidence")
print(f" Correct: {'✓' if pred_class_B == true_label else '✗'}")

if pred_class_A == true_label and pred_class_B == true_label:
    print("\n✓ Both models predicted correctly!")
elif pred_class_A == true_label:
    print("\n✓ Only Model A (With Pooling) predicted correctly!")
elif pred_class_B == true_label:
    print("\n✓ Only Model B (Without Pooling) predicted correctly!")
else:
    print("\n✗ Both models predicted incorrectly!")


